In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')

In [16]:
df = pd.read_csv('IMDB_scraping.csv')
df2 = df.copy()

In [17]:
df2.head()

,Title,Rating,Oscars,Total Nominations and Awards,Director,Gross,MPAA Rating,Users Reviews Count,Critics Reviews Count,Ratings Number
0,American Psycho (2000),7.6,Awards,8 wins & 13 nominations,Mary Harron,"$34,266,564",R,1.3K,324,535K
1,The Thing (1982),8.1,Awards,3 nominations,John Carpenter,"$19,632,053",R,1.2K,374,398K
2,Jaws (1975),8.0,Won 3 Oscars,15 wins & 20 nominations total,Steven Spielberg,"$471,411,300",PG,1.3K,283,569K
3,Heat (1995),8.2,Awards,14 nominations,Michael Mann,"$187,436,818",R,1.2K,225,615K
4,Top Gun (1986),6.9,Won 1 Oscar,11 wins & 5 nominations total,Tony Scott,"$357,288,178",PG,557,208,314K


In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3451 entries, 0 to 3450
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Title                         3451 non-null   object 
 1   Rating                        3451 non-null   float64
 2   Oscars                        3451 non-null   object 
 3   Total Nominations and Awards  3451 non-null   object 
 4   Director                      3451 non-null   object 
 5   Gross                         3451 non-null   object 
 6   MPAA Rating                   3451 non-null   object 
 7   Users Reviews Count           3451 non-null   object 
 8   Critics Reviews Count         3451 non-null   object 
 9   Ratings Number                3451 non-null   object 
dtypes: float64(1), object(9)
memory usage: 269.7+ KB


In [19]:
sum(df2.Title.duplicated())

724

In [20]:
# create a column for number of oscars
import re
df2['Oscar Awards'] = '0'

for i in range(len(df2)): 
    if('won' in df2.Oscars.iloc[i].lower())  :   
        df2['Oscar Awards'][i] = re.findall(r'\d+',df2.Oscars[i])[0]
    else:
        df2['Oscar Awards'][i] = 0

In [54]:
# extrsct the number of total awards wins and nominations
df2['Total Wins'] = '0'
df2['Total Nom'] = '0'
for i in range(len(df2)):
    number =  re.findall(r'\d+',df2['Total Nominations and Awards'][i]) 
    if len(number)==1:
        if('win' in df2['Total Nominations and Awards'][i]):
            df2['Total Wins'][i] = number[0]   
            df2['Total Nom'][i] = 0
        else:
               df2['Total Nom'][i] = number[0]
               df2['Total Wins'][i] = 0
    else:
        
        df2['Total Wins'][i] = number[0]
        df2['Total Nom'][i]  = number[1]
df2['Total Nom']               

0       13
1        3
2       20
3       14
4        5
        ..
3125    28
3126    12
3127     9
3128    11
3129    11
Name: Total Nom, Length: 2523, dtype: int32

In [53]:
#Saparate bewtween the title and the year
df2['Year'] = '0'
for i in range(len(df2)):
    df2['Year'][i] = df2.Title[i][-5:-1]
    df2['Title'][i] = df2.Title[i][0:len(df2['Title'][i])-6]
df2['Title']    

0        American Psycho 
1              The Thing 
2                   Jaws 
3                   Heat 
4                Top Gun 
              ...        
3125              Barfi! 
3126    Paan Singh Tomar 
3127         Delhi Belly 
3128       Peepli [Live] 
3129              Harold 
Name: Title, Length: 2523, dtype: object

In [52]:
# remove symbols and numeric letters
df2['Users Reviews Count'] =df2['Users Reviews Count'].apply(lambda x: x.lower().replace('.','').replace('k','00'))
df2['Critics Reviews Count'] =df2['Critics Reviews Count'].apply(lambda x: str(x).replace('.','').replace('K','00'))
df2['Ratings Number'] =df2['Ratings Number'].apply(lambda x: x.lower().replace('.','').replace('k','00').replace('m','00000'))
df2['Gross'] = df2['Gross'].apply(lambda x: x.replace(',','').replace('$',''))
df2.sample(5)

,Title,Rating,Director,Gross,MPAA Rating,Users Reviews Count,Critics Reviews Count,Ratings Number,Oscar Awards,Total Wins,Total Nom,Year,After 2002
3073,Senritsu meikyû 3D,3.8,Bong Joon Ho,3233364.0,R,162,228,6000,0,44,46,2009,1
2801,Atlas Shrugged: Part I,5.6,Paul Johansson,4627375.0,PG-13,334,64,1400,0,1,0,2011,1
146,The Accountant,7.3,Gavin O'Connor,155160045.0,R,608,341,28100,0,3,5,2016,1
3070,Ah Fei jing juen,7.5,Kar-Wai Wong,3106052.0,Not Rated,53,70,2100,0,17,9,1990,0
1575,Of Mice and Men,7.5,Gary Sinise,5471088.0,PG-13,219,35,4300,0,0,1,1992,0


In [26]:
#Extract numerical values only from the gross column
for i in range(len(df2)):
 numeric = re.findall(r'\d+',df2.Gross[i])
 if len(numeric) ==1:
      df2.Gross[i]= numeric[0]
 if len(numeric) ==0:
    df2.drop(labels=i,axis =0,inplace=True)
df2.reset_index(drop=True, inplace=True)    

In [28]:
#drop date rows from Gross column
for i in range(len(df2)):
    try:
        int(df2.Gross[i])
    except:
        df2.drop(labels=i,axis =0,inplace=True)
df2.reset_index(drop=True, inplace=True)

In [30]:
#change columns to numeric
df2[['Rating','Gross']] = df2[['Rating','Gross']].astype(float)
df2[['Oscar Awards','Total Wins','Total Nom','Year','Critics Reviews Count','Users Reviews Count','Ratings Number']]=df2[['Oscar Awards','Total Wins','Total Nom','Year','Critics Reviews Count','Users Reviews Count','Ratings Number']].astype(int)

In [40]:
df2.drop(['Total Nominations and Awards','Oscars'],axis=1, inplace=True)

In [33]:
df2.drop_duplicates(inplace=True);

In [44]:
#create a boolean column for movies after 2001
df2['After 2002'] = df2['Year'].apply(lambda x: 1 if x >=2003 else 0)

In [49]:
df2.sample(10)

,Title,Rating,Director,Gross,MPAA Rating,Users Reviews Count,Critics Reviews Count,Ratings Number,Oscar Awards,Total Wins,Total Nom,Year,After 2002
447,Léon,8.5,Luc Besson,19552639.0,R,1300,150,1100000,0,6,15,1994,0
2745,Enough Said,7.0,Nicole Holofcener,25288872.0,PG-13,174,247,6100,0,8,38,2013,1
146,The Accountant,7.3,Gavin O'Connor,155160045.0,R,608,341,28100,0,3,5,2016,1
508,Death Wish,7.0,Michael Winner,22000000.0,R,244,113,3800,0,0,2,1974,0
677,Sicario,7.6,Denis Villeneuve,85057064.0,R,826,553,40200,0,15,158,2015,1
1549,Made,6.4,Jon Favreau,5480653.0,R,179,73,1800,0,0,1,2001,0
2661,While You Were Sleeping,6.7,Jon Turteltaub,182057016.0,PG,232,71,9700,0,4,5,1995,0
131,Hercules,7.3,Ron Clements,252712101.0,G,241,95,21800,0,9,16,1997,0
2469,Wonder,8.0,Stephen Chbosky,306209289.0,PG,547,242,15500,0,4,28,2017,1
2180,Menace II Society,7.5,Albert Hughes,27912072.0,R,139,37,5600,0,2,4,1993,0


In [48]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2523 entries, 0 to 3129
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  2523 non-null   object 
 1   Rating                 2523 non-null   float64
 2   Director               2523 non-null   object 
 3   Gross                  2523 non-null   float64
 4   MPAA Rating            2523 non-null   object 
 5   Users Reviews Count    2523 non-null   int32  
 6   Critics Reviews Count  2523 non-null   int32  
 7   Ratings Number         2523 non-null   int32  
 8   Oscar Awards           2523 non-null   int32  
 9   Total Wins             2523 non-null   int32  
 10  Total Nom              2523 non-null   int32  
 11  Year                   2523 non-null   int32  
 12  After 2002             2523 non-null   int64  
dtypes: float64(2), int32(7), int64(1), object(3)
memory usage: 207.0+ KB


In [50]:
df2.to_csv(r'C:\Users\yazan\OneDrive\Desktop\Data Science\Web Scraping\IMDB_data_cleaned.csv',index=False)